In [1]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv

In [2]:
json_path = '/media/salim/DD 2To/GAINED_NIFTI/json/dataset_1.json'
dataset_1 = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset_1.append(info)

print(len(dataset_1))

140


In [3]:
#index
coronal = [8]
error = [80, 135]
false_mask = [13, 14, 17, 18, 19, 39, 42, 47, 48, 51, 53, 54, 57, 58, 59, 61, 62, 63, 64, 65, 67, 68, 69, 74, 76, 86, 88, 97, 100, 108, 111, 112, 113, 115, 117, 119, 122, 124, 128, 132, 133, 134, 136, 137, 138, 139]
print(len(false_mask))

46


In [4]:

good = [5, 10, 12,14, 16, 17, 18, 22, 25, 26, 28,29, 30, 31, 32, 33, 38, 39, 40, 41, 42, 45]
false_mask[45]

139

In [5]:
print(len(good))
data = []
for i in good : 
    data.append(false_mask[i])


for index in data : 
    false_mask.remove(index)

print(len(false_mask))


22
24


In [6]:
serie_mip = []
for index in false_mask : 
    serie_mip.append(dataset_1[index])

In [7]:
len(serie_mip)

24

In [8]:
write_json_file('/media/salim/DD 2To/GAINED_NIFTI','dataset_1_serie_false_mask', serie_mip)

In [9]:
mip_directory = '/media/salim/DD 2To/GAINED_MIP'
csv_directory = '/media/salim/DD 2To/GAINED_CSV_DATA/Export_Gained/CSV41/PET0'
nifti_directory = '/media/salim/DD 2To/GAINED_NIFTI'

mask_false = []

for serie in serie_mip: 
        print(serie_mip.index(serie))
        if serie[1] == 'PT' : 
                serie_pt_objet = SeriesPT(serie[0])
                
                nifti_array = serie_pt_objet.get_numpy_array()

                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
            

                size_matrix = serie_pt_objet.get_size_matrix()
                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
                mask_4D = mask_objet.mask_array

                number_roi = mask_4D.shape[3]
                threshold = mask_objet.details_rois['SUVlo']
                        #seuillage
                mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)

                if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                        print("MASK CORRECT")
                        #generation nifti PT
                        filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                        serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                        print("EXPORT NIFTI PT")
                        #generation nifti CT
                        serie_ct_objet = SeriesCT(serie[2])
                        serie_ct_objet.get_numpy_array()
                        filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                        serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                        print("EXPORT NIFTI CT")
                        #generation nifti mask
                        filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                        serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                        print("EXPORT NIFTI MASK")
                else : 
                        print("FALSE MASK")
                        mask_false.append(serie)
                        print(mask_objet.calcul_suv(nifti_array))
                        print(mask_objet.ecart_suv_max(nifti_array))
                        print(mask_objet.ecart_suv_mean(nifti_array))
                        print(mask_objet.ecart_SD(nifti_array))
                
                        #mip.png MASK
        else  : 
                serie_pt_objet = SeriesPT(serie[2])

                nifti_array = serie_pt_objet.get_numpy_array()

                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
 
                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET


                size_matrix = serie_pt_objet.get_size_matrix()
                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
                mask_4D = mask_objet.mask_array

                number_roi = mask_4D.shape[3]
                threshold = mask_objet.details_rois['SUVlo']
                        #seuillage
                mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)

                if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                        print("MASK CORRECT")
                        #generation nifti PT
                        filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                        serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                        print("EXPORT NIFTI PT")
                        #generation nifti CT
                        serie_ct_objet = SeriesCT(serie[2])
                        serie_ct_objet.get_numpy_array()
                        filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                        serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                        print("EXPORT NIFTI CT")
                        #generation nifti mask
                        filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                        serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                        print("EXPORT NIFTI MASK")
                else : 
                        print("FALSE MASK")
                        mask_false.append(serie)
                        print(mask_objet.calcul_suv(nifti_array))
                        print(mask_objet.ecart_suv_max(nifti_array))
                        print(mask_objet.ecart_suv_mean(nifti_array))
                        print(mask_objet.ecart_SD(nifti_array))

0
taille mask :  (144, 144, 276, 15)
FALSE MASK
{1: {'SUV_max': 4.46, 'SUV_mean': 3.06, 'SD': 0.47}, 2: {'SUV_max': 4.61, 'SUV_mean': 3.7, 'SD': 0.3}, 3: {'SUV_max': 4.86, 'SUV_mean': 3.09, 'SD': 0.67}, 4: {'SUV_max': 7.3, 'SUV_mean': 4.21, 'SD': 0.72}, 5: {'SUV_max': 8.02, 'SUV_mean': 5.16, 'SD': 0.92}, 6: {'SUV_max': 14.22, 'SUV_mean': 7.55, 'SD': 1.51}, 7: {'SUV_max': 9.42, 'SUV_mean': 5.43, 'SD': 1.02}, 8: {'SUV_max': 11.28, 'SUV_mean': 6.32, 'SD': 1.23}, 9: {'SUV_max': 7.68, 'SUV_mean': 4.46, 'SD': 0.98}, 10: {'SUV_max': 5.39, 'SUV_mean': 3.08, 'SD': 0.7}, 11: {'SUV_max': 2.98, 'SUV_mean': 2.03, 'SD': 0.39}, 12: {'SUV_max': 2.99, 'SUV_mean': 1.6, 'SD': 0.34}, 13: {'SUV_max': 5.26, 'SUV_mean': 3.1, 'SD': 0.68}, 14: {'SUV_max': 3.1, 'SUV_mean': 1.95, 'SD': 0.49}, 15: {'SUV_max': 8.75, 'SUV_mean': 5.17, 'SD': 1.24}}
[11, 'POLYGON', 0.1299999999999999]
[]
[]
1
taille mask :  (200, 200, 481, 9)
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
2
taille mask :  (144, 144, 2

taille mask :  (144, 144, 254, 13)
FALSE MASK
{1: {'SUV_max': 20.86, 'SUV_mean': 13.56, 'SD': 2.12}, 2: {'SUV_max': 13.58, 'SUV_mean': 10.02, 'SD': 1.53}, 3: {'SUV_max': 21.37, 'SUV_mean': 13.15, 'SD': 1.88}, 4: {'SUV_max': 10.22, 'SUV_mean': 6.25, 'SD': 1.83}, 5: {'SUV_max': 18.99, 'SUV_mean': 11.93, 'SD': 1.99}, 6: {'SUV_max': 9.04, 'SUV_mean': 5.54, 'SD': 1.24}, 7: {'SUV_max': 4.35, 'SUV_mean': 3.07, 'SD': 0.36}, 8: {'SUV_max': 8.42, 'SUV_mean': 5.2, 'SD': 1.34}, 9: {'SUV_max': 7.19, 'SUV_mean': 4.39, 'SD': 1.14}, 10: {'SUV_max': 10.06, 'SUV_mean': 5.96, 'SD': 1.46}, 11: {'SUV_max': 6.85, 'SUV_mean': 3.86, 'SD': 0.91}, 12: {'SUV_max': 14.4, 'SUV_mean': 8.79, 'SD': 2.46}, 13: {'SUV_max': 12.53, 'SUV_mean': 7.31, 'SD': 1.87}}
[]
[12, 'ELLIPSE', 0.14999999999999858]
[]
5
taille mask :  (256, 256, 299, 9)
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
6
taille mask :  (256, 256, 263, 13)
FALSE MASK
{1: {'SUV_max': 23.43, 'SUV_mean': 12.05, 'SD': 2.37}, 2: {'SUV_max': 21.

taille mask :  (168, 168, 479, 33)
FALSE MASK
{1: {'SUV_max': 24.73, 'SUV_mean': 14.72, 'SD': 3.45}, 2: {'SUV_max': 14.39, 'SUV_mean': 7.65, 'SD': 1.85}, 3: {'SUV_max': 6.34, 'SUV_mean': 3.39, 'SD': 0.68}, 4: {'SUV_max': 12.02, 'SUV_mean': 6.41, 'SD': 1.36}, 5: {'SUV_max': 12.46, 'SUV_mean': 7.26, 'SD': 1.92}, 6: {'SUV_max': 8.18, 'SUV_mean': 4.56, 'SD': 0.96}, 7: {'SUV_max': 6.67, 'SUV_mean': 3.99, 'SD': 0.99}, 8: {'SUV_max': 6.85, 'SUV_mean': 3.9, 'SD': 0.83}, 9: {'SUV_max': 6.39, 'SUV_mean': 3.79, 'SD': 0.89}, 10: {'SUV_max': 8.56, 'SUV_mean': 4.73, 'SD': 1.08}, 11: {'SUV_max': 4.04, 'SUV_mean': 2.32, 'SD': 0.54}, 12: {'SUV_max': 30.34, 'SUV_mean': 18.24, 'SD': 3.66}, 13: {'SUV_max': 27.85, 'SUV_mean': 16.73, 'SD': 4.42}, 14: {'SUV_max': 30.53, 'SUV_mean': 18.16, 'SD': 3.53}, 15: {'SUV_max': 25.05, 'SUV_mean': 14.43, 'SD': 4.13}, 16: {'SUV_max': 11.69, 'SUV_mean': 6.14, 'SD': 1.44}, 17: {'SUV_max': 21.8, 'SUV_mean': 11.6, 'SD': 2.82}, 18: {'SUV_max': 25.19, 'SUV_mean': 13.27, 'SD': 

taille mask :  (144, 144, 255, 28)
FALSE MASK
{1: {'SUV_max': 18.72, 'SUV_mean': 10.57, 'SD': 2.28}, 2: {'SUV_max': 20.57, 'SUV_mean': 11.44, 'SD': 2.37}, 3: {'SUV_max': 20.57, 'SUV_mean': 11.3, 'SD': 2.19}, 4: {'SUV_max': 21.27, 'SUV_mean': 11.64, 'SD': 2.18}, 5: {'SUV_max': 21.43, 'SUV_mean': 11.82, 'SD': 2.2}, 6: {'SUV_max': 14.0, 'SUV_mean': 8.25, 'SD': 2.19}, 7: {'SUV_max': 15.32, 'SUV_mean': 8.7, 'SD': 1.89}, 8: {'SUV_max': 18.69, 'SUV_mean': 10.99, 'SD': 2.3}, 9: {'SUV_max': 17.08, 'SUV_mean': 9.64, 'SD': 2.19}, 10: {'SUV_max': 18.26, 'SUV_mean': 11.04, 'SD': 2.75}, 11: {'SUV_max': 16.92, 'SUV_mean': 9.95, 'SD': 2.3}, 12: {'SUV_max': 14.23, 'SUV_mean': 8.04, 'SD': 2.2}, 13: {'SUV_max': 24.71, 'SUV_mean': 14.48, 'SD': 3.06}, 14: {'SUV_max': 13.12, 'SUV_mean': 7.87, 'SD': 1.76}, 15: {'SUV_max': 8.2, 'SUV_mean': 5.46, 'SD': 1.46}, 16: {'SUV_max': 9.73, 'SUV_mean': 6.06, 'SD': 1.41}, 17: {'SUV_max': 12.8, 'SUV_mean': 5.97, 'SD': 0.71}, 18: {'SUV_max': 7.72, 'SUV_mean': 4.79, 'SD': 0

In [10]:
write_json_file('/media/salim/DD 2To/GAINED_NIFTI','dataset_1_serie_false_mask', mask_false)